Before running this script, make sure that the metafeatures and metadataset files exist in the TabZilla directory;
 from running `tabzilla_results_aggrergator.py`


In [42]:
from pathlib import Path
import pandas as pd
from analysis_utils import get_tuned_alg_perf

metadata_folder = Path("../TabZilla")
use_time_columns = False

# suffix = "_clean" # Downloaded from Google Drive
# suffix = "_drop" 
# suffix = "_lt_min"
# suffix = "_fps_mean"
# suffix = "_broad_mean"
# suffix = "_broad_lt_min"
suffix = "_broad_drop"

metadataset_df = pd.read_csv(metadata_folder / f"metadataset{suffix}.csv")


# TODO:
# Investigate group sparsity structured injection
# Investigate NOT MCAR injection

# read metafeatures
metafeatures_df = pd.read_csv(Path(f"../TabZilla/metafeatures_clean.csv"))

# get the number of instances for each dataset, we will use these later
num_instances = metafeatures_df.loc[:, ["dataset_name", "f__pymfe.general.nr_inst"]]
num_instances.columns = ["dataset_fold_id", "num_inst"]

# make sure that the cleaned_results folder exists
output_folder = Path("./cleaned_results")
output_folder.mkdir(exist_ok=True)    

In [43]:
# merge in num instances - for runtime calculation
metadataset_df = metadataset_df.merge(num_instances, on="dataset_fold_id", how="left")
if use_time_columns:

    # calculate runtime
    time_col = "time__train"

    time_per_inst_col = "train_per_1000_inst"

    metadataset_df.loc[:, time_per_inst_col] = 1000. * metadataset_df[time_col] / metadataset_df["num_inst"]
else:
    print("not using time columns")

not using time columns


### Print number of results per dataset and alg

In [44]:
# for each alg, for how many datasets are there results?
print(f"for each alg: number of datasets with results (out of {len(metadataset_df['dataset_name'].unique())})")
print(metadataset_df.groupby("alg_name")["dataset_name"].apply(lambda x: len(set(x))).sort_values())

print(f"for each dataset: number of algs with results (out of {len(metadataset_df['alg_name'].unique())})")
metadataset_df.groupby("dataset_name")["alg_name"].apply(lambda x: len(set(x))).sort_values()

for each alg: number of datasets with results (out of 1)
alg_name
XGBoost               1
rtdl_FTTransformer    1
rtdl_ResNet           1
Name: dataset_name, dtype: int64
for each dataset: number of algs with results (out of 3)


dataset_name
openml__ada_agnostic__3896    3
Name: alg_name, dtype: int64

# A. Dataset inclusion/exclusion

**In this notebook: selected-18-algs:**
* We use a list of 18 algs (excluding 3 that had lots of errors.)
* We take only the datasets where each of these algs produce a result. This is ~100 datasets.

In [45]:
print(f"for each dataset: number of algs with results")
alg_counts = metadataset_df.groupby("dataset_name")["alg_name"].agg(lambda x: len(set(x))).sort_values()
print(alg_counts.head())

for each dataset: number of algs with results
dataset_name
openml__ada_agnostic__3896    3
Name: alg_name, dtype: int64


In [46]:
# drop:
# - TabPFN (only runs for small datasets)
# - NAM (lots of errors, long runtime)
# - DeepFM (not implemented for multi-class)
# - TabTransformer (lots of bugs...)

# selected_algs = [
#     "SAINT",
#     "NODE",
#     "SVM",
#     "DANet",
#     "rtdl_FTTransformer",
#     "VIME",
#     "STG",
#     "CatBoost",
#     "LightGBM",
#     "KNN",
#     "LinearModel",
#     "TabNet",
#     "RandomForest",
#     "XGBoost",
#     "rtdl_ResNet",
#     "MLP",
#     "DecisionTree",
#     "rtdl_MLP",
#     "NAM",
#     "TabPFNModel", # not included in this version
#     "DeepFM",
#     "TabTransformer", 
# ]

# test_df = metadataset_df.loc[metadataset_df["alg_name"].isin(selected_algs), :]

# # keep only datasets where all selected algs produce a result
# alg_count = test_df.groupby("dataset_name")["alg_name"].apply(lambda x: len(set(x)))

# keep_datasets = alg_count[alg_count == len(selected_algs)].index

# print(f"keeping {len(keep_datasets)} datasets")

# keep_df = test_df.loc[test_df["dataset_name"].isin(keep_datasets), :]

keep_df = metadataset_df

### Slightly more data prep

Note: We will keep all algs, regardless of how many datasets they have results for.

In [47]:
analysis_df = keep_df

print("after removing datasets: number of datasets with results")
dataset_counts = analysis_df.groupby("alg_name")["dataset_name"].agg(lambda x: len(set(x))).sort_values()
print(dataset_counts)

after removing datasets: number of datasets with results
alg_name
XGBoost               1
rtdl_FTTransformer    1
rtdl_ResNet           1
Name: dataset_name, dtype: int64


# B. Tune and rank algorithms for each dataset

**Note**: At this point, you should have a dataframe called `analysis_df`, which contains all results you want to include in the remainder of the analysis. 

The code below performs hyperparameter tuning & ranking of each alg, and writes four cleaned results files to the directory `./cleaned_results`. 

In [48]:
metric_list = [
    "Accuracy",
    "F1",
    "Log Loss",
    "AUC",
]

obj_type_list = [
    "maximize",
    "maximize",
    "minimize",
    "maximize",
]
result_df_dict = {}

### Bookkeeping

In [49]:
# replace alg name with display name
from analysis_utils import ALG_DISPLAY_NAMES, ALG_TYPES
analysis_df.loc[:, "alg_name"] = analysis_df["alg_name"].apply(lambda x: ALG_DISPLAY_NAMES[x])

# add alg type
analysis_df.loc[:, "alg_type"] = analysis_df["alg_name"].apply(lambda x: ALG_TYPES[x])

In [50]:
# add a copy of each "default" hparam row, to treat this as a separate alg
default_rows = analysis_df.loc[analysis_df["hparam_source"] == "default"].copy()
default_rows.loc[:, "alg_name"] = default_rows["alg_name"].apply(lambda x: x + " (default)")

# remove TabPFN and LinearModel, since these only have one hparam set
default_rows = default_rows.loc[~(default_rows["alg_name"].str.contains("TabPFNModel") | default_rows["alg_name"].str.contains("LinearModel")), :]

# append these to the metadataset
analysis_df_with_default = pd.concat([analysis_df, default_rows], ignore_index=True)

### produce cleaned results files

first, each algorithm is tuned for each dataset fold (10 folds per dataset), using all available hyperparameter samples. we then calculate the normalized and ranked performance for each algorithm over all datasets.

the following loop produces four files:
- `./cleaned_results/tuned_aggregated_results.csv`: performance of each tuned algorithm on each dataset, where performance is averaged over all 10 folds. 
- `./cleaned_results/tuned_fold_results.csv`: performance of each tuned algorithm on each dataset fold.
- `./cleaned_results/tuned_aggregated_results_with_default.csv`: same as `tuned_aggregated_results_with_default.csv`, but with the default hyperparameters of each dataset included as a separate algorithm
- `./cleaned_results/tuned_fold_results_with_default.csv`: same as `tuned_fold_results_with_default.csv`, but with the default hyperparameters of each dataset included as a separate algorithm

In [51]:
print(f"use_time_columns: {use_time_columns}")
tuned_result_dfs = {}
for drop_default in [True, False]:
    for i, (metric, objective_type) in enumerate(zip(metric_list, obj_type_list)):

        test_metric_col = metric + "__test"

        if drop_default:
            df = analysis_df.copy()
        else:
            df = analysis_df_with_default.copy()

        tuned_alg_perf = get_tuned_alg_perf(df, metric=metric)

        # NOTE: this "tunes" each algorithm for each training fold separately. so each of the 10 folds might use different hparams.
        tuned_result_dfs[metric] = tuned_alg_perf

        ##############################
        ### STEP 1: TREAT EACH FOLD AS SEPARATE DATASET

        result_col = test_metric_col
        
        # for each dataset, find the min and max metrics over all tuned algs
        overall_bounds = tuned_alg_perf.groupby("dataset_fold_id").agg({result_col: ["min", "max"]}).reset_index()

        # rename the multiindex cols
        new_cols = []
        for c in overall_bounds.columns:
            if c[1] == "":
                new_cols.append(c[0])
            else:
                new_cols.append("_".join(c))

        overall_bounds.columns = new_cols

        tuned_alg_perf = tuned_alg_perf.merge(overall_bounds, on="dataset_fold_id", how="left")

        # add normalized metric
        tuned_alg_perf.loc[:, "normalized_" + result_col] = (tuned_alg_perf[result_col] - tuned_alg_perf[result_col + "_min"]) / (tuned_alg_perf[result_col + "_max"] - tuned_alg_perf[result_col + "_min"])

        # rank all algs for each dataset
        ascending = False if objective_type == "maximize" else True
        
        tuned_alg_perf.loc[:, f"{metric}_rank"] = tuned_alg_perf.groupby(["dataset_fold_id"])[result_col].rank(method="min", ascending=ascending).values

        # keep these cols to merge
        merge_cols = [
            "alg_name", 
            "dataset_fold_id", 
            "normalized_" + result_col,
            f"{metric}_rank",
            result_col + "_min",
            result_col + "_max"
        ]

        if i == 0:
            fold_tuned_df = tuned_alg_perf.copy()
        else:
            fold_tuned_df = fold_tuned_df.merge(tuned_alg_perf[merge_cols], on=["alg_name", "dataset_fold_id"])

        ##############################
        ### STEP 2: AVERAGE OVER FOLDS

        if i == 0:
            agg_dict = {
                test_metric_col: ["median", "mean"],
                # "dataset_name": ["count"],
            }
            if use_time_columns:
                agg_dict["time__train"] = ["median", "mean"]
                agg_dict[time_per_inst_col] = ["median", "mean"]
        else:
            agg_dict = {
                test_metric_col: ["median", "mean"],
            }
            if use_time_columns:
                agg_dict[time_per_inst_col] = ["median", "mean"]

        # aggregate over folds: take the mean & median performance over each fold
        agg_tuned_alg_perf = tuned_alg_perf.groupby(["alg_name", "dataset_name"]).agg(agg_dict).reset_index()
        
        # rename the multiindex cols
        new_cols = []
        for c in agg_tuned_alg_perf.columns:
            if c[1] == "":
                new_cols.append(c[0])
            else:
                new_cols.append("_".join(c))

        agg_tuned_alg_perf.columns = new_cols


        # define the target metric column, we will use this value for all plots
        result_col = test_metric_col + "_mean"

        # for each dataset, find the min and max metrics over all tuned algs
        overall_bounds = agg_tuned_alg_perf.groupby("dataset_name").agg({result_col: ["min", "max"]}).reset_index()
        
        # # normalize metric using these bounds, and get stddev of the mean normalized metric 
        # tuned_alg_perf = tuned_alg_perf.merge(overall_bounds, on="dataset_name", how="left")
        # tuned_alg_perf.loc[:, "norm_metric_tmp"] = (tuned_alg_perf[test_metric_col] -  tuned_alg_perf[result_col + "_min"]) / (tuned_alg_perf[result_col + "_max"] - tuned_alg_perf[result_col + "_min"])
        # std_metric = tuned_alg_pref.groupby() ...

        # rename the multiindex cols
        new_cols = []
        for c in overall_bounds.columns:
            if c[1] == "":
                new_cols.append(c[0])
            else:
                new_cols.append("_".join(c))

        overall_bounds.columns = new_cols

        
        agg_tuned_alg_perf = agg_tuned_alg_perf.merge(overall_bounds, on="dataset_name", how="left")

        # add normalized metric
        agg_tuned_alg_perf.loc[:, "normalized_" + result_col] = (agg_tuned_alg_perf[result_col] - agg_tuned_alg_perf[result_col + "_min"]) / (agg_tuned_alg_perf[result_col + "_max"] - agg_tuned_alg_perf[result_col + "_min"])

        ###### - new - ######
        # estimate the standard deviation of the normalized metric by:
        # 1) normalize the metric for all folds using the normalization here
        # 2) get std of metric over all folds, like we did to get mean and median above
        tmp_df = tuned_alg_perf.loc[:, ["dataset_name", "alg_name", test_metric_col]].copy()
        tmp_df = tmp_df.merge(overall_bounds, on="dataset_name", how="left")    
        tmp_df.loc[:, "normalized_" + test_metric_col] = (tmp_df[test_metric_col] - tmp_df[result_col + "_min"]) / (tmp_df[result_col + "_max"] - tmp_df[result_col + "_min"])
        tmp_agg_df = tmp_df.groupby(["dataset_name", "alg_name"]).agg({"normalized_" + test_metric_col: "std"}).reset_index()
        tmp_agg_df.columns = ["dataset_name", "alg_name", "normalized_" + test_metric_col + "_std"]
        agg_tuned_alg_perf = agg_tuned_alg_perf.merge(tmp_agg_df, on=["dataset_name", "alg_name"], how="left")

        # rank all algs for each dataset
        ascending = False if objective_type == "maximize" else True
        
        # rank according to mean performance over all folds
        agg_method = "mean"

        # rank everything
        agg_tuned_alg_perf.loc[:, f"{metric}_rank_{agg_method}"]  = \
            agg_tuned_alg_perf.groupby(["dataset_name"])[test_metric_col + "_" + agg_method].rank(method="min", ascending=ascending).values

        if use_time_columns:
            agg_tuned_alg_perf.rename(columns={
                time_per_inst_col + "_median": time_per_inst_col + "_median_" + metric,
                time_per_inst_col + "_mean": time_per_inst_col + "_mean_" + metric,
            }, inplace=True)

        # keep these cols to merge
        merge_cols = [
            "alg_name", 
            "dataset_name",
            "normalized_" + result_col,
            "normalized_" + test_metric_col + "_std",
            f"{metric}_rank_mean",
            result_col,
            result_col + "_min",
            result_col + "_max"
        ]
        if use_time_columns:
            merge_cols.extend([
                time_per_inst_col + "_median_" + metric,
                time_per_inst_col + "_mean_" + metric,
            ])

        if i == 0:
            tuned_agg_df = agg_tuned_alg_perf.copy()
        else:
            tuned_agg_df = tuned_agg_df.merge(agg_tuned_alg_perf[merge_cols], on=["alg_name", "dataset_name"])

    # save results

    # merge in alg type, for bookkeeping
    alg_type_df = analysis_df[["alg_name", "alg_type"]].drop_duplicates()
    tuned_agg_df = tuned_agg_df.merge(alg_type_df, on="alg_name", how="left")
    fold_tuned_df = fold_tuned_df.merge(alg_type_df, on="alg_name", how="left")

    if drop_default:
        agg_df_no_default = tuned_agg_df.copy()
        agg_df_no_default.to_csv(f"./cleaned_results/tuned_aggregated_results{suffix}.csv")

        tuned_fold_df_no_default = fold_tuned_df.copy()
        tuned_fold_df_no_default.to_csv(f"./cleaned_results/tuned_fold_results{suffix}.csv")
       
    else:
        agg_df_with_default = tuned_agg_df.copy()
        agg_df_with_default.to_csv(f"./cleaned_results/tuned_aggregated_results_with_default{suffix}.csv")

        tuned_fold_df_with_default = fold_tuned_df.copy()
        tuned_fold_df_with_default.to_csv(f"./cleaned_results/tuned_fold_results_with_default{suffix}.csv")


use_time_columns: False


In [52]:
# take a peek
tuned_agg_df.head()

,alg_name,dataset_name,Accuracy__test_median,Accuracy__test_mean,Accuracy__test_mean_min,Accuracy__test_mean_max,normalized_Accuracy__test_mean,normalized_Accuracy__test_std,Accuracy_rank_mean,normalized_F1__test_mean,...,Log Loss__test_mean,Log Loss__test_mean_min,Log Loss__test_mean_max,normalized_AUC__test_mean,normalized_AUC__test_std,AUC_rank_mean,AUC__test_mean,AUC__test_mean_min,AUC__test_mean_max,alg_type
0,FTTransformer,openml__ada_agnostic__3896,0.846664,0.845686,0.840201,0.856207,0.342707,0.774169,3.0,0.342707,...,0.342836,0.31997,0.350035,0.000000,1.253046,6.0,0.893854,0.893854,0.904811,neural
1,FTTransformer (default),openml__ada_agnostic__3896,0.843372,0.841957,0.840201,0.856207,0.109759,0.729683,4.0,0.109759,...,0.344140,0.31997,0.350035,0.108495,0.973922,5.0,0.895043,0.893854,0.904811,NaN
2,ResNet,openml__ada_agnostic__3896,0.844471,0.841515,0.840201,0.856207,0.082087,0.821446,5.0,0.082087,...,0.337093,0.31997,0.350035,0.326925,1.412536,3.0,0.897436,0.893854,0.904811,neural
3,ResNet (default),openml__ada_agnostic__3896,0.841009,0.840201,0.840201,0.856207,0.000000,0.795460,6.0,0.000000,...,0.350035,0.31997,0.350035,0.152281,1.489668,4.0,0.895522,0.893854,0.904811,NaN
4,XGBoost,openml__ada_agnostic__3896,0.859649,0.855110,0.840201,0.856207,0.931465,0.798244,2.0,0.931465,...,0.320624,0.31997,0.350035,0.760369,1.593638,2.0,0.902185,0.893854,0.904811,gbdt


In [41]:
metafeatures_df["dataset_fold_id"].shape

(1760,)

# merge in metafeatures and rewrite files

In [28]:
import pandas as pd
from pathlib import Path

metafeatures_df = pd.read_csv(Path(f"../TabZilla/metafeatures_clean.csv"))

metafeatures_df.rename(columns={"dataset_name": "dataset_fold_id"}, inplace=True)

metafeatures_df.loc[:, "f__pymfe.general.total_num_instances"] = metafeatures_df["f__pymfe.general.nr_inst"] / 0.8
metafeatures_df.loc[:, "dataset_basename"] = metafeatures_df["dataset_fold_id"].str[:-len("__fold_1")]
agg_metafeatures = metafeatures_df.groupby("dataset_basename").median(numeric_only=True)

# remove histogram columns
keep_cols = [c for c in agg_metafeatures.columns if "hist" not in c]

# rename each column to be easier
# new_col_names = {
#     c: c.removeprefix("f__pymfe.") for c in agg_metafeatures.columns
# }
# agg_metafeatures.rename(columns=new_col_names, inplace=True)

round_attrs = [
    "f__pymfe.general.total_num_instances",
    "f__pymfe.general.nr_attr",
    "f__pymfe.general.nr_bin",
    "f__pymfe.general.nr_cat",
    "f__pymfe.general.nr_num",
    "f__pymfe.general.nr_class",
]
for attr in round_attrs:
    agg_metafeatures.loc[:, attr] = agg_metafeatures[attr].round(0)

agg_metafeatures.to_csv(output_folder / f"agg_metafeatures{suffix}.csv")


AttributeError: 'DataFrame' object has no attribute 'str'

In [29]:
agg_metafeatures["f__pymfe.general.nr_inst"].describe()

NameError: name 'agg_metafeatures' is not defined